Import Necessary Modules

In [ ]:
from operator import itemgetter
from helpers import get_llm, get_retriever
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import Runnable, RunnablePassthrough, RunnableParallel, RunnableLambda, chain
from langchain_core.tools import tool
from typing import Annotated
import requests
from langchain.callbacks.tracers import ConsoleCallbackHandler

#### Define Functions and Schema
In langchain, schema can be defined together with the actual function.

In [2]:
@tool("get_weather")
def get_weather(
    location: Annotated[str, "Location for the weather forecast, e.g. London, UK"]
):
    """Forecast the weather for the provided location."""

    api_key = "777c42660156447db5842748240110"
    result = requests.get(
        f"https://api.weatherapi.com/v1/current.json?key={api_key}&q={location}"
    )

    return result.json()

@tool("get_interest_rate")
def get_interest_rate(
    amount: Annotated[int, "Amount of deposit"],
    interest_rate: Annotated[float, "Interest rate percentage"],
    term: Annotated[int, "Maturity period in month"],
):
    """Interest calculation for fixed deposit."""
    
    interest = amount * (interest_rate / 100) * (term / 12)
    if term > 36:
        interest += 111

    return interest

Create Tool Collection

In [102]:
tools = [get_weather, get_interest_rate]

Initiate Retriever

In [85]:
retriever = get_retriever(
    index_name="test",
    embedding_model="text-embedding-3-large",
    dimension=256,
    vector_db="qdrant",
    top_k=1,
)

Initiate LLM

In [111]:
llm = get_llm("gpt-4o")
llm_with_tools = llm.bind_tools(tools)

Create Instructions for RAG Chain

In [112]:
contextualize_instructions = """Convert the latest user question into a standalone question given the chat history. Don't answer the question, return the question and nothing else (no descriptive text)."""
contextualize_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_instructions),
        ("placeholder", "{chat_history}"),
        ("human", "{question}"),
    ]
)
contextualize_question = contextualize_prompt | llm | StrOutputParser()

qa_instructions = (
    """Use tool calls if necessary. Answer the user question given the following context:\n\n{context}."""
)
qa_prompt = ChatPromptTemplate.from_messages(
    [("system", qa_instructions), ("human", "{question}")]
)

Create a RAG Chain Combined with Tool Calling

In [113]:
@chain
def tool_call(input_: dict) -> Runnable:
    llm_result = input_.get("llm_result")
    print(llm_result)
    if llm_result.tool_calls:
        test_instruction = """Answer the question using the tool response."""
        test_prompt = ChatPromptTemplate.from_messages(
            [("system", test_instruction), ("human", "{question}")]
        )
        test_prompt.messages.append(llm_result)
        for tool_call in llm_result.tool_calls:
            selected_tool = next(
                temp_tool for temp_tool in tools if temp_tool.name == tool_call["name"]
            )
            tool_response = selected_tool.invoke(tool_call)
            test_prompt.messages.append(tool_response)
        return {"question": itemgetter("question")} | test_prompt | llm_with_tools

    else:
        return llm_result

@chain
def contextualize_if_needed(input_: dict) -> Runnable:
    if input_.get("chat_history"):
        return contextualize_question
    else:
        return RunnablePassthrough() | itemgetter("question")

# Pass input query to retriever
retrieve_docs_chain = itemgetter("question") | retriever

def format_docs(docs):
    return "".join(doc.page_content for doc in docs)

formatted_prompt = {
        "question": itemgetter("question") | RunnablePassthrough(),
        "context": lambda x: format_docs(x["context"]),
    } | RunnableParallel(prompt=qa_prompt, question=itemgetter('question'))


llm_result_chain = formatted_prompt | RunnableParallel(llm_result=itemgetter('prompt') | llm_with_tools, question=itemgetter("question"))

output_chain = llm_result_chain | tool_call | StrOutputParser()

final_chain = (
    RunnablePassthrough.assign(question=contextualize_if_needed)
    .assign(context=retrieve_docs_chain)
    .assign(answer=output_chain)
)

# final_chain.get_graph().print_ascii()

Invoke the Chain

In [ ]:
# question = "How much interest can i get if I put in 50000RM for 48 months in fixed deposit account?"
question = "How is the weather today in Yangon?"
result = final_chain.invoke(
    {
        "question": question,
        "chat_history": [],
    }
    , config={"callbacks": [ConsoleCallbackHandler()]}
)
chat_history = []
chat_history.append(("human", question))
chat_history.append(("ai", result["answer"]))
context = result["context"]
chat_history, context

Generate Potential Questions

In [ ]:
llm = get_llm(model_name="meta-llama/Meta-Llama-3-70B-Instruct", temperature=0.9)
prompt = """Generate {k} possible follow up questions based on the given chat history and context. You should follow the instructions below.

- Don't answer the questions.
- Don't include any introductory text, explanations, or follow-up sentences.
- Don't number the question list.
- Keep the questions short and direct. 
- Only generate contextually answerable questions.
- List the questions in a single line, separated by commas without whitespaces.

Example response: What is a cat?,How many legs do they have?
---------------------------------------------------
Chat History: {chat_history}
Context: {context}
"""
prompt_template = ChatPromptTemplate.from_template(prompt)

def get_chat_history(input_):
  return "\n".join([f"{role}: {content}" for role, content in input_])

def get_context(input_):
  return input_[0].page_content

result = {
  "chat_history": itemgetter("chat_history") | RunnableLambda(get_chat_history), 
  "context": itemgetter('context') | RunnableLambda(get_context),
  "k": itemgetter('k')
} | prompt_template | llm | StrOutputParser()
result.invoke({
  "chat_history": chat_history,
  "context": context,
})